In [ ]:
%load_ext dotenv
%dotenv

In [1]:
from rdflib import Graph, URIRef
from rdflib.namespace import Namespace, OWL, RDF, RDFS

from ir_onto_demos_utilities import show_graph_portion

In [2]:
pizza_bisou_g = Graph()
pizza_ns = Namespace("http://www.msesboue.org/o/pizza-data-demo/bisou#")
pizza_bisou_g.bind(prefix="pizza", namespace=pizza_ns)
pizza_bisou_g.parse("./data/pizzas_bisou.ttl")

<Graph identifier=N225ef07d131b4b86866bfa1227b919a6 (<class 'rdflib.graph.Graph'>)>

In [3]:
ingredients_g = Graph()
pizza_ns = Namespace("http://www.msesboue.org/o/pizza-data-demo/bisou#")
ingredients_g.bind(prefix="pizza", namespace=pizza_ns)
ingredients_g.parse("./data/pizzas_bisou_ingredients.ttl")

<Graph identifier=Nd1094b005e5046cb90ddfe4b216c124b (<class 'rdflib.graph.Graph'>)>

In [4]:
map_to_ir_onto_g = Graph()
pizza_ns = Namespace("http://www.msesboue.org/o/pizza-data-demo/bisou#")
ir_onto_ns = Namespace("http://www.msesboue.org/o/ir-ontology#")
map_to_ir_onto_g.bind(prefix="pizza", namespace=pizza_ns)
map_to_ir_onto_g.bind(prefix="ir-onto", namespace=ir_onto_ns)
# map_to_ir_onto_g.parse("./data/pizzas_bisou_map_to_ir_onto.ttl")

In [5]:
# Make the triples actual named individual and object properties
query = """
    CONSTRUCT {
        ?s   rdf:type    owl:NamedIndividual .
        ?p   rdf:type    owl:ObjectProperty .
        ?o   rdf:type    owl:NamedIndividual .
    } WHERE {
        ?s  ?p  ?o .
    }
"""

for triple in pizza_bisou_g.query(query):
    map_to_ir_onto_g.add(triple)

In [6]:
# We have 5 types of instances: Pizza, PizzaTopping, PizzaBase, Spiciness, and PizzaKind
map_to_ir_onto_g.add((pizza_ns.Pizza, RDF.type, OWL.Class))
map_to_ir_onto_g.add((pizza_ns.PizzaTopping, RDF.type, OWL.Class))
map_to_ir_onto_g.add((pizza_ns.PizzaBase, RDF.type, OWL.Class))
map_to_ir_onto_g.add((pizza_ns.Spiciness, RDF.type, OWL.Class))
map_to_ir_onto_g.add((pizza_ns.PizzaKind, RDF.type, OWL.Class))

<Graph identifier=Naaff780e81b14da3851c4f82741d2ab9 (<class 'rdflib.graph.Graph'>)>

In [7]:
# Let us now add the instances of each classes
# Note that we could use reasoning by simply declaring a domain and a range for each object properties
# But it a way more restrictive approach that we might not want

q_construct_pizza_base = """
    CONSTRUCT {
        ?pizza   rdf:type    pizza:Pizza .
        ?base   rdf:type    pizza:PizzaBase .
    } WHERE {
        ?pizza  pizza:has_base  ?base .
    }
"""
for triple in pizza_bisou_g.query(q_construct_pizza_base):
    map_to_ir_onto_g.add(triple)

q_construct_pizza_kind = """
    CONSTRUCT {
        ?pizza   rdf:type    pizza:Pizza .
        ?kind   rdf:type    pizza:PizzaKind .
    } WHERE {
        ?pizza  pizza:has_kind  ?kind .
    }
"""
for triple in pizza_bisou_g.query(q_construct_pizza_kind):
    map_to_ir_onto_g.add(triple)

q_construct_pizza_top = """
    CONSTRUCT {
        ?pizza   rdf:type    pizza:Pizza .
        ?topping   rdf:type    pizza:PizzaTopping .
    } WHERE {
        ?pizza  pizza:has_topping  ?topping .
    }
"""
for triple in pizza_bisou_g.query(q_construct_pizza_top):
    map_to_ir_onto_g.add(triple)

q_construct_pizza_spice = """
    CONSTRUCT {
        ?pizza   rdf:type    pizza:Pizza .
        ?spiciness   rdf:type    pizza:Spiciness .
    } WHERE {
        ?pizza  pizza:has_spiciness  ?spiciness .
    }
"""
for triple in pizza_bisou_g.query(q_construct_pizza_spice):
    map_to_ir_onto_g.add(triple)

In [8]:
len(pizza_bisou_g)

98

In [9]:
# Let us now add the links to the IR ontology
# In our example we categorise pizzas by country, base, kind, topping and spiciness
map_to_ir_onto_g.add((pizza_ns.PizzaBase, RDFS.subClassOf, ir_onto_ns.Category))
map_to_ir_onto_g.add((pizza_ns.PizzaKind, RDFS.subClassOf, ir_onto_ns.Category))
map_to_ir_onto_g.add((pizza_ns.PizzaTopping, RDFS.subClassOf, ir_onto_ns.Category))
map_to_ir_onto_g.add((pizza_ns.Spiciness, RDFS.subClassOf, ir_onto_ns.Category))

# In our example the search engine goal is to find pizzas
map_to_ir_onto_g.add((pizza_ns.Pizza, RDFS.subClassOf, ir_onto_ns.Document))

# In our example when a category is selected we want ot have the subcategories enabled
map_to_ir_onto_g.add((ir_onto_ns.hasSubcategory, RDFS.subPropertyOf, ir_onto_ns.enablesCategory))
map_to_ir_onto_g.add((ir_onto_ns.hasSubcategory, RDF.type, OWL.TransitiveProperty)) # otherwise, only the first subcategory level is enabled

# In our example the relations in our data graph categorizing the pizzas are has_topping, has_kind, has_spiciness, has_base.
map_to_ir_onto_g.add((ir_onto_ns.has_spiciness, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))
map_to_ir_onto_g.add((ir_onto_ns.has_kind, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))
map_to_ir_onto_g.add((ir_onto_ns.has_spiciness, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))
map_to_ir_onto_g.add((ir_onto_ns.has_base, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))

<Graph identifier=Naaff780e81b14da3851c4f82741d2ab9 (<class 'rdflib.graph.Graph'>)>

In [10]:
map_to_ir_onto_g.serialize(destination="./data/pizzas_bisou_map_to_ir_onto.ttl", format="turtle")

<Graph identifier=Naaff780e81b14da3851c4f82741d2ab9 (<class 'rdflib.graph.Graph'>)>

Let us add a transversal relation

In [11]:
# Ex: if a pizza is categorised by a meat topping the category meaty pizza should be enabled
# mapping_graph.add((pizza_onto_demo_ns.makesIt, RDFS.subPropertyOf, ir_onto_ns.enablesCategory))

NameError: name 'mapping_graph' is not defined